In [1]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import col
from delta import *

builder = SparkSession.builder.appName("MyApp") \
    .config("spark.sql.extensions", "io.delta.sql.DeltaSparkSessionExtension") \
    .config("spark.sql.catalog.spark_catalog", "org.apache.spark.sql.delta.catalog.DeltaCatalog")

spark = configure_spark_with_delta_pip(builder).getOrCreate()

your 131072x1 screen size is bogus. expect trouble
23/08/29 17:27:05 WARN Utils: Your hostname, Pramod resolves to a loopback address: 127.0.1.1; using 172.23.91.132 instead (on interface eth0)
23/08/29 17:27:05 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address


:: loading settings :: url = jar:file:/root/miniconda3/envs/pramod_dev/lib/python3.10/site-packages/pyspark/jars/ivy-2.5.1.jar!/org/apache/ivy/core/settings/ivysettings.xml


Ivy Default Cache set to: /root/.ivy2/cache
The jars for the packages stored in: /root/.ivy2/jars
io.delta#delta-core_2.12 added as a dependency
:: resolving dependencies :: org.apache.spark#spark-submit-parent-e4db3721-077d-4f4e-a3a6-3b2b9cafd5c1;1.0
	confs: [default]
	found io.delta#delta-core_2.12;2.4.0 in central
	found io.delta#delta-storage;2.4.0 in central
	found org.antlr#antlr4-runtime;4.9.3 in central
:: resolution report :: resolve 167ms :: artifacts dl 5ms
	:: modules in use:
	io.delta#delta-core_2.12;2.4.0 from central in [default]
	io.delta#delta-storage;2.4.0 from central in [default]
	org.antlr#antlr4-runtime;4.9.3 from central in [default]
	---------------------------------------------------------------------
	|                  |            modules            ||   artifacts   |
	|       conf       | number| search|dwnlded|evicted|| number|dwnlded|
	---------------------------------------------------------------------
	|      default     |   3   |   0   |   0   |   0  

In [5]:
import random, string
df = spark.createDataFrame([[i, random.choice(string.ascii_uppercase)] for i in range(100)], ['id', 'value'])

In [9]:
df.write.format('delta').saveAsTable("test_delta_table")

23/08/29 17:34:22 WARN MemoryManager: Total allocation exceeds 95.00% (1,020,054,720 bytes) of heap memory
Scaling row group sizes to 95.00% for 8 writers
23/08/29 17:34:22 WARN MemoryManager: Total allocation exceeds 95.00% (1,020,054,720 bytes) of heap memory
Scaling row group sizes to 84.44% for 9 writers
23/08/29 17:34:22 WARN MemoryManager: Total allocation exceeds 95.00% (1,020,054,720 bytes) of heap memory
Scaling row group sizes to 76.00% for 10 writers
23/08/29 17:34:22 WARN MemoryManager: Total allocation exceeds 95.00% (1,020,054,720 bytes) of heap memory
Scaling row group sizes to 69.09% for 11 writers
23/08/29 17:34:22 WARN MemoryManager: Total allocation exceeds 95.00% (1,020,054,720 bytes) of heap memory
Scaling row group sizes to 63.33% for 12 writers
23/08/29 17:34:22 WARN MemoryManager: Total allocation exceeds 95.00% (1,020,054,720 bytes) of heap memory
Scaling row group sizes to 58.46% for 13 writers
23/08/29 17:34:22 WARN MemoryManager: Total allocation exceeds 95.

In [10]:
spark.sql('select * from test_delta_table').show()

+---+-----+
| id|value|
+---+-----+
| 72|    N|
| 73|    B|
| 74|    O|
| 75|    B|
| 76|    O|
| 77|    A|
| 84|    Y|
| 85|    G|
| 86|    Z|
| 87|    K|
| 88|    C|
| 89|    C|
|  6|    I|
|  7|    K|
|  8|    X|
|  9|    G|
| 10|    F|
| 11|    W|
| 30|    Y|
| 31|    G|
+---+-----+
only showing top 20 rows



In [11]:
spark.sql('select * from test_delta_table').count()

100

In [12]:
spark.sql('DELETE from test_delta_table where id = 1')

DataFrame[num_affected_rows: bigint]

In [13]:
spark.sql('select * from test_delta_table').count()

99

In [14]:
spark.sql('OPTIMIZE test_delta_table ZORDER BY id')

DataFrame[path: string, metrics: struct<numFilesAdded:bigint,numFilesRemoved:bigint,filesAdded:struct<min:bigint,max:bigint,avg:double,totalFiles:bigint,totalSize:bigint>,filesRemoved:struct<min:bigint,max:bigint,avg:double,totalFiles:bigint,totalSize:bigint>,partitionsOptimized:bigint,zOrderStats:struct<strategyName:string,inputCubeFiles:struct<num:bigint,size:bigint>,inputOtherFiles:struct<num:bigint,size:bigint>,inputNumCubes:bigint,mergedFiles:struct<num:bigint,size:bigint>,numOutputCubes:bigint,mergedNumCubes:bigint>,numBatches:bigint,totalConsideredFiles:bigint,totalFilesSkipped:bigint,preserveInsertionOrder:boolean,numFilesSkippedToReduceWriteAmplification:bigint,numBytesSkippedToReduceWriteAmplification:bigint,startTimeMs:bigint,endTimeMs:bigint,totalClusterParallelism:bigint,totalScheduledTasks:bigint,autoCompactParallelismStats:struct<maxClusterActiveParallelism:bigint,minClusterActiveParallelism:bigint,maxSessionActiveParallelism:bigint,minSessionActiveParallelism:bigint>,de

In [15]:
spark.sql('DESCRIBE HISTORY test_delta_table').show()

+-------+--------------------+------+--------+--------------------+--------------------+----+--------+---------+-----------+-----------------+-------------+--------------------+------------+--------------------+
|version|           timestamp|userId|userName|           operation| operationParameters| job|notebook|clusterId|readVersion|   isolationLevel|isBlindAppend|    operationMetrics|userMetadata|          engineInfo|
+-------+--------------------+------+--------+--------------------+--------------------+----+--------+---------+-----------+-----------------+-------------+--------------------+------------+--------------------+
|      2|2023-08-29 17:34:...|  null|    null|            OPTIMIZE|{predicate -> [],...|null|    null|     null|          1|SnapshotIsolation|        false|{numRemovedFiles ...|        null|Apache-Spark/3.4....|
|      1|2023-08-29 17:34:...|  null|    null|              DELETE|{predicate -> ["(...|null|    null|     null|          0|     Serializable|        fa

In [16]:
spark.sql('SELECT count(*) from test_delta_table VERSION AS OF 0').show()

+--------+
|count(1)|
+--------+
|     100|
+--------+



In [17]:
spark.sql('SELECT count(*) from test_delta_table VERSION AS OF 1').show()

+--------+
|count(1)|
+--------+
|      99|
+--------+



In [18]:
spark.sql('VACUUM test_delta_table').show()

Deleted 0 files and directories in a total of 1 directories.
+--------------------+
|                path|
+--------------------+
|file:/root/Work/R...|
+--------------------+



In [19]:
spark.conf.set('spark.databricks.delta.retentionDurationCheck.enabled', False)
spark.sql('VACUUM test_delta_table').show()

Deleted 0 files and directories in a total of 1 directories.
+--------------------+
|                path|
+--------------------+
|file:/root/Work/R...|
+--------------------+



In [20]:
spark.conf.set('spark.databricks.delta.retentionDurationCheck.enabled', False)
spark.sql('VACUUM test_delta_table RETAIN 0 HOURS').show()

Deleted 17 files and directories in a total of 1 directories.
+--------------------+
|                path|
+--------------------+
|file:/root/Work/R...|
+--------------------+

